<a href="https://colab.research.google.com/github/heoraon/Machine-Leaning/blob/main/Week6_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install statsmodels

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import scipy as sp
import matplotlib as mpl

In [3]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=3148bd6c345be62fa6b24d96232b9dafd9f5754a70060cc96f80de5f932241a2
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [4]:
import wget

In [5]:
url1 = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt'
wget.download(url1)

'ratings_train.txt'

In [6]:
url2 = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt'
wget.download(url2)

'ratings_test.txt'

In [7]:
import codecs

with codecs.open("ratings_train.txt", encoding='utf-8') as f:
  data = [line.split('\t') for line in f.read().splitlines()]
  data = data[1:] #header 제외

In [8]:
from pprint import pprint
pprint(data[0])

['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']


In [9]:
X = list(zip(*data))[1]
y = np.array(list(zip(*data))[2], dtype=int)

In [10]:
# 데이터를 다항 나이브 베이즈 모형으로 학습(6주차 영상 참고)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer #벡터화 도구
from sklearn.naive_bayes import MultinomialNB #학습 모델
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report #성능 평가 도구

model1 = Pipeline([
    ('vect', CountVectorizer()),
    ('mb', MultinomialNB()),
])

In [12]:
model1.fit(X, y)

Pipeline(steps=[('vect', CountVectorizer()), ('mb', MultinomialNB())])

In [13]:
import codecs
with codecs.open("ratings_test.txt", encoding='utf-8') as f:
  data_test = [line.split('\t') for line in f.read().splitlines()]
  data_test = data_test[1:]

In [14]:
X_test = list(zip(*data_test))[1]
y_test = np.array(list(zip(*data_test))[2], dtype=int)

print(classification_report(y_test, model1.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83     24827
           1       0.84      0.81      0.82     25173

    accuracy                           0.83     50000
   macro avg       0.83      0.83      0.83     50000
weighted avg       0.83      0.83      0.83     50000



In [15]:
# Tfidf 방법을 사용했을 때와 비교(벡터화 방법 변경)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

model2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('mb', MultinomialNB()),
])

In [17]:
model2.fit(X, y)

Pipeline(steps=[('vect', TfidfVectorizer()), ('mb', MultinomialNB())])

In [18]:
print(classification_report(y_test, model2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83     24827
           1       0.84      0.81      0.83     25173

    accuracy                           0.83     50000
   macro avg       0.83      0.83      0.83     50000
weighted avg       0.83      0.83      0.83     50000



In [19]:
# 형태소 분석기를 사용한 결과와 비교

In [20]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 38.6 MB/s eta 0:00:00


In [22]:
from konlpy.tag import Okt
pos_tagger = Okt()

def tokenize_pos(doc):
  return ['/'.join(t) for t in pos_tagger.pos(doc)]

In [23]:
model3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_pos)),
    ('mb', MultinomialNB())
])

In [24]:
model3.fit(X, y)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize_pos at 0x78398e39b560>)),
                ('mb', MultinomialNB())])

In [ ]:
print(classification_report(y_test, model3.predict(X_test)))

In [ ]:
# (1,2)-gram 을 사용하면 성능이 더 개선

In [ ]:
model4 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1, 2))),
    ('mb', MultinomialNB()),
])

In [ ]:
model4.fit(X, y)

In [ ]:
print(classification_report(y_test, model4.predict(X_test)))